# Export the mean of the SD maps

In [39]:
# import the libraries
import ee
import pandas as pd
import os
import numpy as np
import random
from random import sample
import itertools 
import geopandas as gpd
from sklearn.metrics import r2_score
from termcolor import colored # this is allocate colour and fonts type for the print title and text
from IPython.display import display, HTML

In [40]:
# initialize the earth engine API
ee.Initialize()

In [41]:
# load the basic maps that needed for the analysis
# load the carbon concentration map
carbonConcentration = ee.Image("users/leonidmoore/ForestBiomass/Biome_level_Wood_Carbon_Conentration_Map")
# load the two composites tha will be used in the analysis
compositeImage =ee.Image("users/leonidmoore/ForestBiomass/20200915_Forest_Biomass_Predictors_Image")
compositeImageNew = ee.Image("projects/crowtherlab/Composite/CrowtherLab_Composite_30ArcSec")
# load the present forest cover
presentForestCover = compositeImage.select('PresentTreeCover')
# load the biome layer 
biomeLayer = compositeImage.select("WWF_Biome")
# define a pixel area layer with unit km2
pixelAreaMap = ee.Image.pixelArea().divide(10000);
# define the boundary geography reference
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False)

## load all the SD maps from Arnan et al., Spawn et al., and Walker et al.

In [42]:
carbonDensityWK =  ee.Image("users/leonidmoore/ForestBiomass/WalkerMap/reprojected_Walker_map_1km")
# since the SD map from Arnan is biomass density, therefore, we directly transfer thre biomass density into carbon density by multiplying the carbon concentration
carbonDensitySD =  ee.Image("users/leonidmoore/ForestBiomass/RemoteSensingModel/ESA_CCI_AGB_Map_bias_corrected_1km_2010").multiply(carbonConcentration).mask(presentForestCover.gt(0))
carbonDensityHM =  ee.Image("users/leonidmoore/ForestBiomass/SpawnMap/Spawn_Harmonized_AGB_density_Map_1km").select('agb')

In [43]:
# compose the  images into a multiband image
carbonDensityImage = carbonDensityWK.addBands(carbonDensitySD).addBands(carbonDensityHM)
# apply the reducer 
carbonDensityMean = carbonDensityImage.reduce(ee.Reducer.mean()).mask(presentForestCover.gt(0))

In [44]:
exportTask = ee.batch.Export.image.toCloudStorage(image = carbonDensitySD,
                                                  description = 'Satellite_carbon_stock_density_mean_map_Export',
                                                  fileNamePrefix = 'ForestBiomassExport/SD_Arnan_carbon_stock_density_mean_Map',
                                                  region = unboundedGeo,
                                                  bucket = "crowtherlab_gcsb_lidong",
                                                  crs = 'EPSG:4326',
                                                  crsTransform = [0.008333333333333333,0,-180,0,-0.008333333333333333,90],
                                                  maxPixels = 1e13,
                                                  fileFormat = 'GeoTIFF')
# start the export task
exportTask.start()
# show the task status
exportTask.status()


{'state': 'READY',
 'description': 'Satellite_carbon_stock_density_mean_map_Export',
 'creation_timestamp_ms': 1670614986478,
 'update_timestamp_ms': 1670614986478,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'AZT64NR3EHNM56RWHJGRA6OH',
 'name': 'projects/earthengine-legacy/operations/AZT64NR3EHNM56RWHJGRA6OH'}